In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm as cm
import matplotlib.image
import matplotlib.path
import matplotlib.patches
import matplotlib.ticker
from matplotlib import animation
import matplotlib.gridspec as gridspec
import h5py
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import svgutils.transform as sg
import scipy.signal
import scipy.stats

from pathlib import Path
import re

import itertools
sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

import rushd as rd

In [ ]:
main_palette = {
    'tandem': '#225A9B',
    'tandem_alt': '#19D2BF',
    'convergent': '#FFB133',
    'divergent': '#FE484E',
    'gray': '#666666',
    'light_gray': '#888888',
    'axis_gray': '#262626'
}
font_sizes = {
    'colorbar_title': 8,
    'subaxis_title': 9,
    'line_annotation': 8,
    'data_annotation': 8,
}
light_palette = {k:v + '28' for k, v in main_palette.items()}
no_yellow_viridis = matplotlib.colors.ListedColormap(cm.get_cmap('viridis', 256)(np.linspace(0,0.8,256)))
k_formatter = matplotlib.ticker.FuncFormatter(lambda x, _: f'{x:.0f}' if abs(x) < 1000 else f'{x/1000:.0f}k')
fold_formatter = matplotlib.ticker.FuncFormatter(lambda x, _: f'{x}x')
hours_formatter = matplotlib.ticker.FuncFormatter(lambda x,_: f'{x/3600:.0f}')
hours_1f_formatter = matplotlib.ticker.FuncFormatter(lambda x,_: f'{x/3600:.1f}')
kb_formatter = matplotlib.ticker.FuncFormatter(lambda x,_: f'{x/0.34/1000:.1f}')

In [ ]:
plt.rcParams['animation.ffmpeg_path'] = r'C:\Users\ChemeGrad2019\Downloads\ffmpeg-2020-12-09-git-7777e5119a-full_build\bin\ffmpeg.exe'
#sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})
ffmpeg_writer = animation.FFMpegWriter(fps=30, codec='libx264', extra_args=['-crf', '18', '-pix_fmt', 'yuv420p'])
movie_examples = h5py.File(rd.infile(rd.rootdir/'output'/'seminar_examples.h5'))

In [ ]:
raw_points = '-1,0 -1,0.5 -0.1,0.5 0,0.5 0.1,0.5 1,0.5 1,0 1,-0.5 0.1,-0.5 0,-0.5 -0.1,-0.5 -1,-0.5 -1,0 0,0'
points = np.array([(float(point.split(',')[0]), float(point.split(',')[1])) for point in raw_points.split(' ')])
scaled_points = np.copy(points) * 0.6
scaled_points[:,1] -= 0.5
all_points = np.concatenate((scaled_points, points), axis=0)
all_points[:,1] += 0.15
ellipse_path = matplotlib.path.Path(all_points,
    codes=([matplotlib.path.Path.MOVETO] + [matplotlib.path.Path.CURVE4]*12 + [matplotlib.path.Path.CLOSEPOLY]) * 2
)
def gen_animation(data_group, gene_colors, ex_offset, filename, n_frames, dpi=120, mRNA_remap=None, tspan=None):
    genes = list(zip(data_group.attrs['gene.start'], data_group.attrs['gene.end']))
    gene_directions = [1 - 2 * (gene[0] > gene[1]) for gene in genes]

    times = np.array(data_group['time'])
    mRNA_data = np.array(data_group['mRNA'])
    if tspan is None:
        tspan = (0, max(data_group['time']))
    max_length = data_group.attrs['bcs.length']
    max_gene_length = max([abs(gene[1] - gene[0]) for gene in genes])

    fig = plt.figure(figsize=(1400 / dpi,700 / dpi))
    plot_ax = plt.subplot2grid((2,2), (0,0))
    sigma_ax = plt.subplot2grid((2,2), (1,0))
    mRNA_ax = plt.subplot2grid((2,2), (0,1), rowspan=2)

    plot_ax.set_xlim([0,max_length])
    plot_ax.set_ylim([-20, 100 + max_gene_length])
    plot_ax.spines['top'].set_visible(False)
    plot_ax.spines['right'].set_visible(False)
    plot_ax.spines['left'].set_visible(False)
    plot_ax.tick_params(left=False, labelleft=False)
    plot_ax.set_xticks(np.arange(0.0, 12.5, 2.5) * 1000 * 0.34)
    plot_ax.xaxis.set_major_formatter(matplotlib.ticker.NullFormatter())

    sigma_ax.set_xlim([0,max_length])
    sigma_ax.set_ylim([-0.6, 0.6])
    sigma_ax.spines['top'].set_visible(False)
    sigma_ax.spines['right'].set_visible(False)
    sigma_ax.set_xticks(np.arange(0.0, 12.5, 2.5) * 1000 * 0.34)
    sigma_ax.xaxis.set_major_formatter(kb_formatter)
    sigma_ax.set_xlabel('Distance (kb)')
    sigma_ax.set_ylabel('σ')
    sigma_ax.text(50, 0.4, 'Overwound')
    sigma_ax.text(50, -0.5, 'Underwound')

    mRNA_ax.set_xlim(tspan)
    mRNA_ax.set_ylim([0,np.max(data_group['mRNA'])])
    mRNA_ax.spines['top'].set_visible(False)
    mRNA_ax.spines['right'].set_visible(False)
    mRNA_ax.set_xlabel('Time (hr)')
    mRNA_ax.set_ylabel('# mRNAs')
    mRNA_ax.set_xticks([0, 10000, 3600 * 6])
    mRNA_ax.xaxis.set_major_formatter(hours_1f_formatter)

    fig.tight_layout()

    sigma_ax.axhline(0, color=main_palette['axis_gray'])
    sigma_line = sigma_ax.plot([], [], linewidth=3, color='#3d8c7d')[0]

    mRNA_ax.axvspan(10000, max(times), color=main_palette['light_gray'] + '55')
    if mRNA_remap is None:
        gene_lines = [mRNA_ax.plot([], [], color=color)[0] for gene, color in zip(genes, gene_colors)]
    else:
        gene_lines = [mRNA_ax.plot([], [], color=color)[0] for gene, color in zip(genes, [gene_colors[mRNA_remap[i]] for i in range(len(gene_colors))])]

    xax_transform = plot_ax.get_xaxis_transform()
    for gene, gene_dir, color in zip(genes, gene_directions, gene_colors):
        base = gene[0] - (max_length * 0.34 * 0.01)
        plot_ax.plot([base, base], [20/(120 + max_gene_length), 0.23], linewidth=5, solid_capstyle='projecting', color=color, transform=xax_transform, clip_on=False)
        plot_ax.plot([base, base + gene_dir * (max_length * 0.34 * 0.06)], [0.23, 0.23], linewidth=5, solid_capstyle='projecting', color=color, transform=xax_transform, clip_on=False)
        plot_ax.add_artist(matplotlib.patches.FancyArrow(base + gene_dir * (max_length * 0.34 * 0.06), 0.23, gene_dir, 0,
            width=0.1, head_width=0.1, head_length=max_length * 0.34 * 0.05, color=color, transform=xax_transform, clip_on=False))


    plot_ax.plot([0, max_length], [0, 0], color=main_palette['light_gray']+'55', linewidth=14, zorder=0, solid_capstyle='butt')
    
    gene_plots = [plot_ax.plot(gene, [0, 0], color=color,linewidth=14, zorder=0, solid_capstyle='butt')[0] for gene, color in zip(genes, gene_colors)]

    mRNA_lines = matplotlib.collections.LineCollection([], colors='#d18cca')

    polymerases = plot_ax.plot([], [], marker=ellipse_path, ls='',
                        mec='#3d8c7d', mfc='#82d4bd', mew=2, ms=15, zorder=2.5)[0]

    def init():
        mRNA_lines.set_linewidth(3)
        plot_ax.add_collection(mRNA_lines)
        return (*gene_plots, mRNA_lines)
    
    def animate(i):
        t = tspan[0] + (tspan[1] - tspan[0]) * (i / n_frames)
        possible_idxes = np.where(times < t)[0]
        t_idx = possible_idxes[-1] if len(possible_idxes) > 0 else 0
        frame_vals = {k:data_group[k][:,t_idx] for k in ['rnap_location', 'phi', 'mRNA_length', 'mRNA']}
        n_polymerases = np.sum(frame_vals['rnap_location'] >= 0)

        # Plot mRNA data values

        # Plot mRNA values (on the right-hand plot)
        gene_time_mask = (times < t)
        for i, gene_line in enumerate(gene_lines):
            gene_line.set_data(times[gene_time_mask], mRNA_data[i,gene_time_mask])

        # Set visibility
        polymerases.set_visible(n_polymerases > 0)
        mRNA_lines.set_visible(n_polymerases > 0)

        # Plot polymerases and genes coming off polymerases.
        if n_polymerases > 0:
            rnap_locs = np.concatenate((
                np.zeros(1,),
                frame_vals['rnap_location'][:n_polymerases],
                np.ones(1,) * max_length
            ))
            phi = np.concatenate((
                np.zeros(1,),
                frame_vals['phi'][:n_polymerases],
                np.zeros(1,)
            ))
            sigma = np.diff(phi) / np.diff(rnap_locs) / -1.85

            # Set supercoiling line
            sigma_line.set_data(
                np.concatenate(([rnap_locs[0]], [loc for loc in rnap_locs[1:-1] for _ in range(2)], [rnap_locs[-1]])),
                [s for s in sigma for _ in range(2)]
            )

            points = np.array([rnap_locs, np.zeros_like(rnap_locs)]).T.reshape(-1,1,2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            polymerases.set_data(rnap_locs[1:-1], np.zeros_like(rnap_locs[1:-1]))
            mRNA_base = np.array([rnap_locs[1:-1], np.zeros(rnap_locs[1:-1].shape)]).T.reshape(-1,1,2)
            mRNA_tail = np.array([rnap_locs[1:-1], frame_vals['mRNA_length'][:n_polymerases]]).T.reshape(-1,1,2)
            mRNA_segments = np.concatenate([mRNA_base, mRNA_tail], axis=1)
            mRNA_lines.set_segments(mRNA_segments)
        else:
            sigma_line.set_data([0, max_length], [0,0])
        return [sigma_line, polymerases, mRNA_lines] + list(gene_lines)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=n_frames, interval=33, blit=True)
    anim.save(str(filename), dpi=dpi, writer=ffmpeg_writer, progress_callback= lambda i,n: print(f'Saving frame {i} of {n}') if i % 250 == 0 else None)

In [ ]:
gen_animation(movie_examples['tangles_full_run.000018'], [main_palette['tandem'], main_palette['gray']], 0, rd.outfile(rd.rootdir/'output'/'short_tandem_upstream.mp4'), 15*30, tspan=[0,3600*6], mRNA_remap={0:1, 1:0})
gen_animation(movie_examples['tangles_full_run.000022'], [main_palette['gray'], main_palette['tandem_alt']], 1, rd.outfile(rd.rootdir/'output'/'short_tandem_downstream.mp4'), 15*30, tspan=[0,3600*6])
gen_animation(movie_examples['tangles_full_run.000025'], [main_palette['gray'], main_palette['convergent']], 2, rd.outfile(rd.rootdir/'output'/'short_convergent.mp4'), 15*30, tspan=[0,3600*6])
gen_animation(movie_examples['tangles_full_run.000027'], [main_palette['gray'], main_palette['divergent']], 3, rd.outfile(rd.rootdir/'output'/'short_divergent.mp4'), 15*30, tspan=[0,3600*6])